In [36]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://fit.haui.edu.vn/vn"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
print(docs)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.62it/s]

[Document(metadata={'source': 'https://fit.haui.edu.vn/vn', 'title': 'KHOA CÔNG NGHỆ THÔNG TIN - ĐẠI HỌC CÔNG NGHIỆP HÀ NỘI || HaUI', 'description': 'Khoa Công nghệ thông tin (CNTT) được thành lập ngày 05/07/1999 theo Quyết định số 115/TCHC của Hiệu trưởng Trường Cao Đẳng Công nghiệp Hà Nội. Khi thành lập, Khoa chỉ có 07 cán bộ giáo viên được trang bị hơn 20 máy vi tính với khối lượng công việc lớn và mới. Tập thể cán bộ, giáo viên của Khoa đã tập trung nhiều công sức để xây dựng Khoa. Năm đó, khoa đã tuyển được 6 lớp Trung học và 7 lớp Cao đẳng Tin.', 'language': 'en'}, page_content='\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <meta name="description" content="Khoa C&#244;ng nghệ th&#244;ng tin (CNTT) được th&#224;nh lập ng&#224;y 05/07/1999 theo Quyết định số 115/TCHC của Hi&#234;̣u trưởng Trư

In [19]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

In [26]:
from langchain.chains import create_extraction_chain
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ['GOOGLE_API_KEY'] =os.environ["GEMINI_API_KEY"]

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}


def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

In [ ]:
import pprint

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

def scrape_with_playwright(urls, schema):
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["span"]
    )
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content


urls = ["https://fit.haui.edu.vn/vn"]
extracted_content = scrape_with_playwright(urls, schema=schema)